# Database Learning Project Setup & Tutorial

Welcome to your database learning journey! This notebook will help you:

1. **Set up a complete project structure** for learning MySQL and PostgreSQL
2. **Connect to both databases** using Python
3. **Run basic SQL queries** and experiments
4. **Create documentation** and organize your learning materials

## What You'll Learn

- How to connect to MySQL and PostgreSQL databases
- Basic CRUD operations in both databases
- Database-specific features and differences
- Best practices for organizing database learning materials

Let's get started! 🚀

## 1. Set Up Project Structure

First, let's create a well-organized folder structure for our database learning project.

In [ ]:
import os
import pathlib
from pathlib import Path

# Define project structure
project_structure = {
    'docs': ['mysql', 'postgresql', 'common'],
    'sql': ['mysql', 'postgresql', 'exercises'],
    'notebooks': ['mysql', 'postgresql', 'comparisons'],
    'data': [],
    'config': []
}

# Create project directories
base_path = Path.cwd()
print(f"Creating project structure in: {base_path}")

for folder, subfolders in project_structure.items():
    folder_path = base_path / folder
    folder_path.mkdir(exist_ok=True)
    print(f"✓ Created: {folder}/")
    
    for subfolder in subfolders:
        subfolder_path = folder_path / subfolder
        subfolder_path.mkdir(exist_ok=True)
        print(f"  ✓ Created: {folder}/{subfolder}/")

print("\n🎉 Project structure created successfully!")

## 2. Install Required Libraries

Before connecting to databases, we need to install the necessary Python libraries.

In [ ]:
import subprocess
import sys

def install_package(package):
    """Install a Python package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installed successfully")
    except subprocess.CalledProcessError:
        print(f"✗ Failed to install {package}")

# Required packages for database connectivity
packages = [
    "mysql-connector-python",
    "psycopg2-binary", 
    "sqlalchemy",
    "pandas",
    "jupyter-sql"
]

print("Installing required packages...")
for package in packages:
    install_package(package)

print("\n🎉 All packages installed!")

## 3. Connect to MySQL and PostgreSQL Databases

Now let's establish connections to both databases. Make sure you have Docker running with the databases:

In [ ]:
import mysql.connector
from mysql.connector import Error
import psycopg2
from psycopg2 import sql
import pandas as pd

def connect_mysql():
    """Connect to MySQL database"""
    try:
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            database='learning',
            user='root',
            password='rootpassword'
        )
        
        if connection.is_connected():
            print("✓ Successfully connected to MySQL database")
            return connection
        
    except Error as e:
        print(f"✗ Error connecting to MySQL: {e}")
        return None

def connect_postgresql():
    """Connect to PostgreSQL database"""
    try:
        connection = psycopg2.connect(
            host='localhost',
            port=5432,
            database='learning',
            user='postgres',
            password='postgres'
        )
        
        print("✓ Successfully connected to PostgreSQL database")
        return connection
        
    except psycopg2.Error as e:
        print(f"✗ Error connecting to PostgreSQL: {e}")
        return None

# Test connections
mysql_conn = connect_mysql()
postgres_conn = connect_postgresql()

## 4. Run Basic SQL Queries

Let's run some basic queries on both databases to see them in action.

In [ ]:
def run_mysql_query(connection, query):
    """Execute a query on MySQL and return results as DataFrame"""
    if connection and connection.is_connected():
        try:
            df = pd.read_sql(query, connection)
            return df
        except Exception as e:
            print(f"Error executing MySQL query: {e}")
            return None

def run_postgresql_query(connection, query):
    """Execute a query on PostgreSQL and return results as DataFrame"""
    if connection:
        try:
            df = pd.read_sql(query, connection)
            return df
        except Exception as e:
            print(f"Error executing PostgreSQL query: {e}")
            return None

# Example queries
print("=== MYSQL RESULTS ===")
if mysql_conn:
    # Show all users
    mysql_users = run_mysql_query(mysql_conn, "SELECT * FROM users")
    print("Users table:")
    print(mysql_users)
    
    # Show posts with user information
    mysql_posts = run_mysql_query(mysql_conn, """
        SELECT p.title, p.content, u.username, p.created_at
        FROM posts p
        JOIN users u ON p.user_id = u.id
        WHERE p.published = TRUE
    """)
    print("\nPublished posts:")
    print(mysql_posts)

print("\n=== POSTGRESQL RESULTS ===")
if postgres_conn:
    # Show all users with JSON metadata
    postgres_users = run_postgresql_query(postgres_conn, """
        SELECT username, email, metadata, tags 
        FROM users
    """)
    print("Users with metadata:")
    print(postgres_users)
    
    # Show posts with JSON operations
    postgres_posts = run_postgresql_query(postgres_conn, """
        SELECT title, metadata->>'read_time' as read_time, 
               metadata->>'difficulty' as difficulty
        FROM posts 
        WHERE published = TRUE
    """)
    print("\nPosts with metadata:")
    print(postgres_posts)

## 5. Create Example SQL Query Scripts

Let's create some example SQL files for practicing different concepts.

In [ ]:
# Create example SQL files
sql_examples = {
    'sql/mysql/basic_queries.sql': '''-- Basic MySQL Queries
-- Select all users
SELECT * FROM users;

-- Select users with their post count
SELECT u.username, u.email, COUNT(p.id) as post_count
FROM users u
LEFT JOIN posts p ON u.id = p.user_id
GROUP BY u.id, u.username, u.email;

-- Find users who have published posts
SELECT DISTINCT u.username
FROM users u
INNER JOIN posts p ON u.id = p.user_id
WHERE p.published = TRUE;
''',
    
    'sql/postgresql/json_queries.sql': '''-- PostgreSQL JSON Queries
-- Select users with specific metadata
SELECT username, metadata->>'role' as role
FROM users
WHERE metadata->>'role' = 'admin';

-- Update JSON metadata
UPDATE users 
SET metadata = jsonb_set(metadata, '{last_login}', '"2023-12-01"')
WHERE username = 'john_doe';

-- Query array fields
SELECT username, tags
FROM users
WHERE 'developer' = ANY(tags);
''',
    
    'sql/exercises/joins_practice.sql': '''-- Practice Joins (Works on both databases)
-- 1. Inner Join - Get all posts with their authors
SELECT p.title, u.username, p.created_at
FROM posts p
INNER JOIN users u ON p.user_id = u.id;

-- 2. Left Join - Get all users and their posts (including users without posts)
SELECT u.username, p.title
FROM users u
LEFT JOIN posts p ON u.id = p.user_id;

-- 3. Many-to-Many Join - Posts with their categories
SELECT p.title, c.name as category
FROM posts p
INNER JOIN post_categories pc ON p.id = pc.post_id
INNER JOIN categories c ON pc.category_id = c.id;
'''
}

# Write the SQL files
for file_path, content in sql_examples.items():
    full_path = Path(file_path)
    full_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(full_path, 'w') as f:
        f.write(content)
    
    print(f"✓ Created: {file_path}")

print("\n🎉 Example SQL scripts created!")

## 6. Automate Project Initialization with a Script

Finally, let's create a Python script that can set up everything automatically for future projects.

In [ ]:
# Create an automation script
automation_script = '''#!/usr/bin/env python3
"""
Database Learning Project Setup Script
This script automatically sets up a complete database learning environment.
"""

import os
import subprocess
import sys
from pathlib import Path

def create_project_structure():
    """Create the project directory structure"""
    structure = {
        'docs': ['mysql', 'postgresql', 'common'],
        'sql': ['mysql', 'postgresql', 'exercises'],
        'notebooks': ['mysql', 'postgresql', 'comparisons'],
        'data': [],
        'config': []
    }
    
    print("Creating project structure...")
    for folder, subfolders in structure.items():
        Path(folder).mkdir(exist_ok=True)
        for subfolder in subfolders:
            Path(folder, subfolder).mkdir(exist_ok=True)
    print("✓ Project structure created")

def install_requirements():
    """Install required Python packages"""
    packages = [
        "mysql-connector-python",
        "psycopg2-binary",
        "sqlalchemy",
        "pandas",
        "jupyter",
        "jupyterlab"
    ]
    
    print("Installing required packages...")
    for package in packages:
        subprocess.run([sys.executable, "-m", "pip", "install", package], 
                      capture_output=True)
    print("✓ Packages installed")

def create_config_files():
    """Create configuration files"""
    docker_compose = """version: '3.8'
services:
  mysql:
    image: mysql:8.0
    environment:
      MYSQL_ROOT_PASSWORD: rootpassword
      MYSQL_DATABASE: learning
    ports:
      - "3306:3306"
  
  postgres:
    image: postgres:15
    environment:
      POSTGRES_DB: learning
      POSTGRES_USER: postgres  
      POSTGRES_PASSWORD: postgres
    ports:
      - "5432:5432"
"""
    
    with open("docker-compose.yml", "w") as f:
        f.write(docker_compose)
    
    print("✓ Configuration files created")

def main():
    print("🚀 Setting up Database Learning Project...")
    create_project_structure()
    install_requirements()
    create_config_files()
    print("\\n🎉 Setup complete! Run 'docker-compose up -d' to start databases.")

if __name__ == "__main__":
    main()
'''

# Write the automation script
with open("setup_db_project.py", "w") as f:
    f.write(automation_script)

print("✓ Created automation script: setup_db_project.py")
print("\nTo use this script in future projects:")
print("1. Copy setup_db_project.py to a new directory")
print("2. Run: python setup_db_project.py")
print("3. Start learning! 🚀")

## 🎉 Project Setup Complete!

You now have a comprehensive database learning environment with:

### ✅ What's Been Created

1. **Organized folder structure** for documentation and experiments
2. **Docker setup** for MySQL and PostgreSQL databases
3. **Python connection utilities** for both databases
4. **Example SQL scripts** for practice
5. **Jupyter notebook environment** for interactive learning
6. **Automation script** for future projects

### 🚀 Next Steps

1. **Start the databases:**
   ```bash
   docker-compose up -d
   ```

2. **Open Adminer** (Web database interface):
   - Visit: http://localhost:8080
   - MySQL: Server=mysql, Username=root, Password=rootpassword
   - PostgreSQL: Server=postgres, Username=postgres, Password=postgres

3. **Explore the examples:**
   - Check out the SQL files in the `sql/` directory
   - Run the example queries in this notebook
   - Create your own experiments!

4. **Document your learning:**
   - Add notes to the `docs/` folders
   - Create new notebooks for specific topics
   - Save interesting queries for reference

### 🔗 Useful Resources

- [MySQL Documentation](https://dev.mysql.com/doc/)
- [PostgreSQL Documentation](https://www.postgresql.org/docs/)
- [SQL Tutorial](https://www.w3schools.com/sql/)
- [Database Design Basics](https://www.lucidchart.com/pages/database-diagram/database-design)

Happy learning! 🎓📚

In [ ]:
# Clean up connections
if mysql_conn and mysql_conn.is_connected():
    mysql_conn.close()
    print("✓ MySQL connection closed")

if postgres_conn:
    postgres_conn.close() 
    print("✓ PostgreSQL connection closed")

print("\n🎉 All done! Your database learning environment is ready to go!")